In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import os

In [3]:
import sys

def get_n_dir_up(path, n):
    for _ in range(n):
        path = os.path.dirname(path)
    return path

CUR_PATH= os.path.abspath("__file__")
sys.path.append(os.path.join(get_n_dir_up(CUR_PATH, 2)))

In [4]:
PLOTS_FOLDER = os.path.join(
    get_n_dir_up(CUR_PATH, 3), 'results', 'images', 'behavior', 'hand')

In [5]:
import json

DEFAULT_DATA_FOLDER = os.path.join(
    get_n_dir_up(CUR_PATH, 3), 'data')
with open(os.path.join(DEFAULT_DATA_FOLDER, 'QA.json')) as f:
    qa_records = json.load(f)

invalid_subjs = qa_records['invalid']

In [12]:
DEFAULT_FEATURE_FOLDER = os.path.join(
    get_n_dir_up(CUR_PATH, 2), 'gaze_analysis', 'features')
all_subjs = os.listdir(DEFAULT_FEATURE_FOLDER)
all_subjs = [subj for subj in all_subjs if subj.isdigit()]
all_subjs = [subj for subj in all_subjs if int(subj) not in invalid_subjs]

In [6]:
MOUSE_DATA_FOLDER = os.path.join(
    get_n_dir_up(CUR_PATH, 3), 'data', 'mouse', 'compressed')

### Firstly, get the cleaned mouse data

In [7]:
MOUSE_CLEANED_DATA_FOLDER = os.path.join(
    get_n_dir_up(CUR_PATH, 3), 'data', 'mouse', 'cleaned')

In [8]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

In [16]:
from tqdm import tqdm

from utils.mouse_motion import (
    PHASE_OFFSETS, aggregate_stroke_events, stroke_detected_to_dicts
)

def save_all_cleaned_motions():
    for subj in tqdm(all_subjs):
        subj_f = f'{subj}.npz'
        subj_src_path = os.path.join(MOUSE_DATA_FOLDER, subj_f)
        subj_des_path = os.path.join(MOUSE_CLEANED_DATA_FOLDER, subj_f)
        if not os.path.exists(subj_des_path):
            subj_source_data = np.load(subj_src_path)
            strokes = aggregate_stroke_events(subj_source_data, PHASE_OFFSETS)
            stroke_dicts = stroke_detected_to_dicts(strokes)
            np.savez(subj_des_path, **stroke_dicts)

In [19]:
save_all_cleaned_motions()

100%|██████████| 33/33 [00:00<00:00, 2980.45it/s]


## load pair-up behavior and motion data